# Preparazione dei dati

## Preparazione dei dati
Processo di generazione dei fold illustrato per le 3 tipologie di dataset creati per le 2 lingue:

- Tipo Dataset 1: composto dalla giustapposizione dei set di caratteristiche linguistiche dei 2 saggi;
- Tipo Dataset 2: composto dalla differenza di valori per ogni caratteristica linguistica dei set dei 2 saggi;
- Tipo Dataset 3: composto dalla differenza di valori per ogni caratteristica linguistica e dalla giustapposizione dei set dei 2 saggi.

### Generazione folds - Tipo Dataset 1

In [ ]:
output_dir = '/kfold_DoubleFeat'
# Lettura del dataframe contenente gli ID dei saggi e le feature legate all'ordine
dtype = {'Essay_1': str, 'Essay_2': str}
df= pd.read_csv('Training_CItA.tsv', delimiter="\t",  dtype=dtype)  
# Lettura del DataFrame contenente gli ID dei saggi con annesse feature linguistiche estratte tramite ProfilingUD
df_5497_2 = pd.read_csv('profilingUD/5497.csv', sep='\t')

# Numero di folds
n_splits = 10

# Se non esiste l'output directory la crea
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Inizializzazione dell'oggetto per realizzare la k-fold cross-validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Itera sui fold e divide i dati nei set selezionandone gli indici
for i, (train_index, test_index) in enumerate(kf.split(df)):
    # Crea cartela per il fold corrente
    fold_dir = os.path.join(output_dir, f'fold_{i}')
    if not os.path.exists(fold_dir):
        os.makedirs(fold_dir)

    # Seleziona le righe che corrispondono agli indici per il training e test set
    train_data = df.iloc[train_index]
    test_data = df.iloc[test_index]
    
    # Creo una copia dei dataset di training e test del fold corrente
    df_2_train = train_data.copy()
    df_2_test = test_data.copy()
    
    # Effettuo uno switch dell'ordine dei set di feature 
    # Processo per il training set
    
    # Crea una copia di Essay_1 in una variabile temporanea
    temp = df_2_train['Essay_1'].copy() 
    # Assegna i valori di Essay_2 ad Essay_1
    df_2_train['Essay_1'] = df_2_train['Essay_2'] 
    # Assegna i valori temporanei ad Essay_2
    df_2_train['Essay_2'] = temp 
    
    # Crea una copia di Order_1 in una variabile temporanea
    temp2 = df_2_train['Order_1'].copy() 
    # Assegna i valori di Order_2 ad Order_1
    df_2_train['Order_1'] = df_2_train['Order_2'] 
    # Assegna i valori temporanei ad Order_2
    df_2_train['Order_2'] = temp2 
    #(Processo ripetuto per il test set)
    
    # Ciclo per il training set:
    # Itera sul DataFrame di training contenente gli ID dei saggi e le feature legate all'ordine
    for index, row in df_2_train.iterrows():

        # Costruzione della chiave di merge per l'ID del saggio Essay_1
        key_of_merge = "text_CIItA/" + str(row['Essay_1']) + ".conllu"

        # Cerca la chiave di merge nel dataset contenente le features
        match_row = df_5497_2[df_5497_2['Filename'] == key_of_merge]

        # Se esiste una corrispondenza
        if not match_row.empty:
            
            # Rinomina le colonne di feature contenute in match_row aggiungendo l'indicazione del saggio di appartenenza Essay_1
            match_row = match_row.rename(columns={col: col + '_Essay1' for col in match_row.columns})
           
            # Unisce queste colonne al dataframe contenente gli ID dei saggi e le feature legate all'ordine
            for column in match_row.columns:
                df_2_train.loc[index, column] = match_row.loc[match_row.index[0], column]
                
        # Costruzione della chiave di merge per l'ID del saggio Essay_2
        key_of_merge_2 = "text_CIItA/" + str(row['Essay_2']) + ".conllu"

        # Cerca la chiave di merge nel dataset contenente le features
        match_row_2 = df_5497_2[df_5497_2['Filename'] == key_of_merge_2]

        # Se esiste una corrispondenza
        if not match_row_2.empty:
            
            # Rinomina le colonne di feature contenute in match_row aggiungendo l'indicazione del saggio di appartenenza Essay_2
            match_row_2 = match_row_2.rename(columns={col: col + '_Essay2' for col in match_row_2.columns})
            
            # Unisce queste colonne al dataframe contenente gli ID dei saggi e le feature legate all'ordine
            for column in match_row_2.columns:
                 df_2_train.loc[index, column] = match_row_2.loc[match_row_2.index[0], column]
                 
    # Ciclo per il test set:
    # Assegnazione label 0 agli elementi nel dataframe di training invertito 
    df_2_train['Order_Label'] = 0
    # Concatenazione dei dataframe di training con label 0 e 1 per il fold corrente
    df_train_final = pd.concat([train_data, df_2_train], axis=0, ignore_index=True)   
    
    # Assegnazione ripetuta per test:
    # Salvataggio dei dataframe di training e test del fold corrente
    df_train_final.to_csv(os.path.join(fold_dir, f'train.tsv'), sep='\t', index=False)
    df_test_final.to_csv(os.path.join(fold_dir, f'test.tsv'), sep='\t', index=False)


### Generazione folds - Tipo Dataset 2

In [ ]:
# Definisco numero di fold
num_folds =10
# Ciclo sui fold (processo ripetuto anche per il test set)
for i in range(num_folds):
    # Lettura dei dati di addestramento dai fold dove sono contenuti i set di feature di entrambi i saggi
    train_data = pd.read_csv(f"kfold_DoubleFeat/fold_{i}/train.tsv", sep='\t')

    # Otteniamo il numero totale di righe del DataFrame
    num_rows = train_data.shape[0]
    # Selezioniamo la metà delle righe
    half_rows = num_rows // 2

    # DataFrame dalla prima metà del set (ordine cronologico)
    df1_cronologico = train_data.iloc[:half_rows]  
    # DataFrame dalla seconda metà del set (ordine invertito)
    df2_NOTCronologico = train_data.iloc[half_rows:]  

    # Resetto l'indice sul set di dati con ordine invertito
    df2_NOTCronologico= df2_NOTCronologico.reset_index(drop=True)
    # Elimino colonne dove sono contenute le feature dei rispettivi Essay_2 da entrambi i set
    df1_cronologico = df1_cronologico.drop(columns=df1_cronologico.columns[df1_cronologico.columns.get_loc ('n_sentences_Essay2'):])
    df2_NOTCronologico = df2_NOTCronologico.drop(columns=df2_NOTCronologico.columns[df2_NOTCronologico.columns.get_loc ('n_sentences_Essay2'):])
    
    # Sottraggo blocco di feature di Essay_1 da Essay_2 e appongo la label 1 (ordine cronologico)
    new_train_data_1= df1_cronologico.sub(df2_NOTCronologico)
    new_train_data_1['Order_Label'] = 1
    
    # Sottraggo blocco di feature di Essay_2 da Essay_1 e appongo la label 0 (ordine invertito)
    new_train_data_2= df2_NOTCronologico.sub(df1_cronologico)
    new_train_data_2['Order_Label'] = 0
    
    # Concateno le differenze di set in un unico dataframe
    new_train_data = pd.concat([new_train_data_1, new_train_data_2], axis=0, ignore_index=True) 
      
    # Crea la directory se non esiste
    if not os.path.exists(f"kfold_FeaturesDifference/fold_{i}"):
        os.makedirs(f"kfold_FeaturesDifference/fold_{i}")
        
    # Salva il nuovo DataFrame in un file CSV nella directory corretta
    new_train_data.to_csv(f"kfold_FeaturesDifference/fold_{i}/train.tsv", index=False, sep='\t')

### Generazione folds - Tipo Dataset 3

In [ ]:
# Numero di fold
num_folds = 10

# Cartella in cui salvare i dataframe
output_folder = 'kfold_Combo_and_Difference'
# Crea la cartella di output
os.makedirs(output_folder, exist_ok=True)

# Ciclo sui fold
for i in range(num_folds):
   
    # Percorso delle cartelle kfold_FeaturesDifference e kfold_DoubleFeat
    folder_path_1 = f'kfold_FeaturesDifference/fold_{i}'
    folder_path_2 = f'kfold_DoubleFeat/fold_{i}'
    
    # Crea cartella di output per il fold corrente
    output_fold_folder = os.path.join(output_folder, f"fold_{i}")
    os.makedirs(output_fold_folder, exist_ok=True)
    
    # Carica i dataframe di test e training dal fold corrente di kfold_DoubleFeat
    df_1_test = pd.read_csv(os.path.join(folder_path_1, 'test.tsv'), sep='\t')
    df_1_train = pd.read_csv(os.path.join(folder_path_1, 'train.tsv'), sep='\t')
     # Carica i dataframe di test e training dal fold corrente di kfold_FeaturesDifference
    df_2_test = pd.read_csv(os.path.join(folder_path_2, 'test.tsv'), sep='\t')
    df_2_train = pd.read_csv(os.path.join(folder_path_2, 'train.tsv'), sep='\t')
    
    # Se i valori di Order_Label dei dataframe di training e test corrispondono droppo una delle due colonne di label
    if (df_1_test['Order_Label'] == df_2_test['Order_Label']).all():
        df_2_test = df_2_test.drop('Order_Label', axis=1)
    if (df_1_train['Order_Label'] == df_2_train['Order_Label']).all():
        df_2_train = df_2_train.drop('Order_Label', axis=1)
    
    # Concateno i dataframe di test e training 
    merged_test = pd.concat([df_2_test, df_1_test], axis=1)
    merged_train = pd.concat([df_2_train, df_1_train], axis=1)
    
    # Salva i dataframe combinati in nuovi file CSV nella cartella di output
    merged_test.to_csv(os.path.join(output_fold_folder, "test.tsv"), index=False, sep='\t')
    merged_train.to_csv(os.path.join(output_fold_folder, "train.tsv"), index=False, sep='\t')

## Creazione dei dataset di training, test, validation per l'addestramento di Bert

In [ ]:
# Funzione che prende in input un dataframe e ne restituisce uno arricchito con i record di ordine invertito
def modify_dataframe(df):
    
    # Copia il dataFrame 
    df_copy = df.copy()

    # Inverte le colonne contenenti gli ID dei saggi e dei testi 
    df_copy[['Essay_1', 'Essay_2']] = df_copy[['Essay_2', 'Essay_1']]
    df_copy[['Essay_1_Text', 'Essay_2_Text']] = df_copy[['Essay_2_Text', 'Essay_1_Text']]
    # Crea nuova colonna contenente i due saggi separati dal token [SEP]
    df_copy['Formatted_Text'] = df_copy['Essay_1_Text'] + ' [SEP] ' + df_copy['Essay_2_Text']

    # Assegna il valore 0 agli elementi in Order_Label
    df_copy['Order_Label'] = 0

    # Concatena il DataFrame originale e il DataFrame modificato e lo restituisce
    df_final = pd.concat([df, df_copy], axis=0, ignore_index=True)
    return df_final

# Lettura del dataframe contenente gli ID e i saggi
dtype = {'Essay_1': str, 'Essay_2': str}
df= pd.read_csv('Training_CItA.csv', delimiter=",",  dtype=dtype)  

# Crea una copia del dataframe
train_150 = df.copy()

# Effettua separazione grezza delle stringhe in base agli spazi
train_150['Essay_1_Text'] = train_150['Essay_1_Text'].str.split()
train_150['Essay_2_Text'] = train_150['Essay_2_Text'].str.split()

# Seleziona solo le prime 150 unità per ogni saggio
train_150['Essay_1_Text'] = train_150['Essay_1_Text'].apply(lambda x: ' '.join(x[:150]))
train_150['Essay_2_Text'] = train_150['Essay_2_Text'].apply(lambda x: ' '.join(x[:150]))

# Crea la colonna 'Formatted_Text' contenente i saggi troncati e separati dal token [SEP]
train_150['Formatted_Text'] = train_150['Essay_1_Text'] + ' [SEP] ' + train_150['Essay_2_Text']

# Creo set di test e validation
train_df_150, temp_df_150 = train_test_split(train_150, test_size=0.3, random_state=42)
test_df_150, val_df_150 = train_test_split(temp_df_150, test_size=0.1, random_state=42)

# Chiama la funzione per ottenere i dataframe con i record di ordine invertito
train_with0_150 = modify_dataframe(train_df_150)
test_with0_150 = modify_dataframe(test_df_150)
val_with0_150 = modify_dataframe(val_df_150)

# Salva i dataframe modificato in file CSV
train_with0_150.to_csv('Train_CItA_Finetuning.csv', index=False)
test_with0_150.to_csv('Test_CItA_Finetuning.csv', index=False)
val_with0_150.to_csv('Val_CItA_Finetuning.csv', index=False)